In [ ]:
import os
import numpy as np

data_path = "/home/jxliu/workspace/Code/examples/exp1/data/Total"
arousal_dict, dominance_dict, valence_dict, emotion_dict = {}, {}, {}, {}

for audio_base_folder in ["train"]:
    utt2ADV_file = os.path.join(data_path, audio_base_folder, "utt2ADV")
    utt2emo_flie = os.path.join(data_path, audio_base_folder, "utt2emo")
    with open(utt2ADV_file, 'r', encoding='utf-8') as f1:
        lines = f1.readlines()
        for line in lines:
            # 'utt [arousal, dominance, valence] for each lines'
            parts = line.strip().split(' ', 1)
            utt = parts[0]
            ADV_values = eval(parts[1])  # [arousal, dominance, valence]
            arousal = float(ADV_values[0])  # arousal
            dominance = float(ADV_values[1])  # dominance
            valence = float(ADV_values[2])  # valence
            if arousal == 0 and dominance == 0 and valence == 0:
                continue
            if arousal > 7.0 or dominance > 7.0 or valence > 7.0:
                print(f"{utt} ADV value is more than 7.0")
                continue
            if arousal <1.0 or dominance <1.0 or valence <1.0:
                print(f"{utt} ADV value is less than 1.0")
                continue
            
       
            arousal_dict[utt] = arousal
            dominance_dict[utt] = dominance
            valence_dict[utt] = valence
    with open(utt2emo_flie, 'r', encoding='utf-8') as f2:
        for line in f2:
            parts = line.strip().split(' ')
            utt = parts[0]
            emotion_label = parts[1]
            emotion_dict[utt] = emotion_label

# label list
emotion_labels = sorted(list(set(emotion_dict.values())))
assert arousal_dict.keys() == dominance_dict.keys() == valence_dict.keys()
utts = list(arousal_dict.keys())
data = np.array([[arousal_dict[utt], dominance_dict[utt], valence_dict[utt]] for utt in utts])

In [ ]:
import numpy as np
from collections import defaultdict

# Calculate the ADV center values for each emotion
emotion_groups = defaultdict(list)
for utt in utts:
    emo = emotion_dict[utt]
    point = [arousal_dict[utt], dominance_dict[utt], valence_dict[utt]]
    emotion_groups[emo].append(point)

emotion_centers = {}
for emo, points in emotion_groups.items():
    points_array = np.array(points)
    # Calculate the mean center in three-dimensional space
    center = np.mean(points_array, axis=0)
    emotion_centers[emo] = {
        'center': center,
        'count': len(points)
    }
for emo, info in emotion_centers.items():
    print(f"Emotion {emo} (Samples Count:{info['count']}):")
    print(f"  ADV: {info['center']}")
    
# Calculate global extremum
dimension_ranges = {
    'arousal': (np.min(data[:, 0]), np.max(data[:, 0])),
    'dominance': (np.min(data[:, 1]), np.max(data[:, 1])),
    'valence': (np.min(data[:, 2]), np.max(data[:, 2]))
}
print(f"Arousal: {dimension_ranges['arousal']}")
print(f"Dominance: {dimension_ranges['dominance']}")
print(f"Valence: {dimension_ranges['valence']}")

In [ ]:
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde
import numpy as np

def plot_histogram_with_bins(data, bins, title, xlabel):
    plt.figure(figsize=(10, 6))
    plt.hist(data, bins=29, density=True, alpha=0.7, 
             color='skyblue', edgecolor='black', 
             label='Histogram')
    
    for boundary in bins:
        plt.axvline(x=boundary, color='red', linestyle='--', 
                    linewidth=1.5, alpha=0.8)
    
    kde = gaussian_kde(data)
    x = np.linspace(min(data)-0.1, max(data)+0.1, 1000)
    plt.plot(x, kde(x), color='darkorange', linewidth=2, 
             label='KDE')
    
    stats_text = f"""
    Mean: {np.mean(data):.2f}
    Std: {np.std(data):.2f}
    Min: {np.min(data):.2f}
    Max: {np.max(data):.2f}
    Bins: {len(bins)-1}
    """
    plt.text(0.98, 0.7, stats_text, transform=plt.gca().transAxes,
             ha='right', va='top', fontsize=20,
             bbox=dict(facecolor='white', alpha=0.8))
    
    plt.xlabel(f"{title} with Bins", fontsize=28)
    plt.ylabel('Density', fontsize=28)
    plt.xticks(fontsize=20)
    plt.yticks(fontsize=20)
    plt.grid(True, alpha=0.3)
    plt.legend(loc='upper left', fontsize=20)
    plt.tight_layout()
    plt.savefig(f'pictures/Total_{title}.png')
    plt.cla()

In [ ]:
from sklearn.cluster import KMeans
import numpy as np

def get_cluster_bins(data, n_bins):
    """Generate non-uniform box boundaries"""
    
    kmeans = KMeans(n_clusters=n_bins, random_state=42, n_init=10)
    kmeans.fit(data)
    labels = kmeans.fit_predict(data)
    centers = kmeans.cluster_centers_
    # centers = np.sort(kmeans.cluster_centers_.flatten())
    axis_names = ['a', 'd', 'v']
    axis_boundaries = {}
    internal_boundaries = {}
    cluster_counts = np.array([np.sum(labels == k) for k in range(n_bins)], dtype=int)
    cluster_vars = np.zeros((n_bins, 3), dtype=float)
    for k in range(n_bins):
        members = data[labels == k]
        if members.shape[0] <= 1: 
            cluster_vars[k, :] = 0.0
        else:
            cluster_vars[k, :] = np.var(members, axis=0, ddof=0)
            
    # per-axis boundaries
    for axis_idx, axis in enumerate(axis_names):
        centers_axis = centers[:, axis_idx]   # (K,)
        order = np.argsort(centers_axis) 
        sorted_centers = centers_axis[order]
        t_list = []
        for i in range(n_bins - 1):
            idx_i = order[i]
            idx_j = order[i + 1]
            m_i = float(sorted_centers[i])
            m_j = float(sorted_centers[i + 1])
            # midpoint
            t_mid = 0.5 * (m_i + m_j)
            # weighted
            n_i = int(cluster_counts[idx_i])
            n_j = int(cluster_counts[idx_j])
            t_w = t_mid if (n_i + n_j) == 0 else (n_i * m_i + n_j * m_j) / (n_i + n_j)
            sigma_i2 = float(cluster_vars[idx_i, axis_idx])
            sigma_j2 = float(cluster_vars[idx_j, axis_idx])
            if sigma_i2 == 0.0 and sigma_j2 == 0.0:
                r = 1.0
            else:
                small = min(sigma_i2, sigma_j2)
                large = max(sigma_i2, sigma_j2)
                r = np.inf if small == 0.0 else (large / small)
            t_final = t_w if r > 2.0 else t_mid
            t_list.append(float(t_final))
        internal_boundaries[axis] = np.array(t_list, dtype=float)
        gmin = float(np.min(X[:, axis_idx]))
        gmax = float(np.max(X[:, axis_idx]))
        axis_boundaries[axis] = np.concatenate(([gmin], internal_boundaries[axis], [gmax]))
    boundaries = np.vstack([axis_boundaries[ax] for ax in ['a', 'd', 'v']])
    return boundaries

In [ ]:
# Generate uniform bins
# arousal_bin_boundaries = np.histogram_bin_edges(data[:, 0], bins=14)
# dominance_bin_boundaries = np.histogram_bin_edges(data[:, 1], bins=14)
# valence_bin_boundaries = np.histogram_bin_edges(data[:, 2], bins=14)

# Generate non-uniform bins, n_bins is calculated by the central limit theorem
boundaries = get_cluster_bins(data, n_bins=14)


print("Arousal bin boundaries:", boundaries[0])
print("Dominance bin boundaries:", boundaries[1])
print("Valence bin boundaries:", boundaries[2])

plot_histogram_with_bins(data[:, 0], boundaries[0], 'Arousal Distribution', 'Arousal')
plot_histogram_with_bins(data[:, 1], boundaries[1], 'Dominance Distribution', 'Dominance')
plot_histogram_with_bins(data[:, 2], boundaries[2], 'Valence Distribution', 'Valence')

array_dict = {'arousal_bin':boundaries[0], 
              'dominance_bin':boundaries[1], 
              'valence_bin':boundaries[2]}
np.savez('bins.npz', **array_dict)